# Pre-training Discriminator

In [ ]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 50.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
import keras_core as keras
import keras_nlp

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

Using TensorFlow backend


In [ ]:
df_train = pd.read_csv("train.csv")
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
BATCH_SIZE = 4
VAL_SPLIT = 0.2
EPOCHS = 1

In [ ]:
X = df_train["text"]
y = df_train["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VAL_SPLIT, random_state=42)

In [ ]:
preset = "distil_bert_base_en_uncased"
preset = "bert_tiny_en_uncased"

# Distil
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(preset, sequence_length=160, name="preprocessor_4_tweets")
classifier = keras_nlp.models.BertClassifier.from_preset(preset, preprocessor=preprocessor, num_classes=2)

17602216/17602216 [==============================] - 0s 0us/step


In [ ]:
classifier.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #'binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(1e-5),
    metrics=["accuracy"]
)

In [ ]:
history = classifier.fit(x=X_train,
                         y=y_train,
                         batch_size=BATCH_SIZE,
                         epochs=EPOCHS,
                         validation_data=(X_val, y_val)
                        )

1523/1523 [==============================] - 112s 64ms/step - loss: 0.5965 - accuracy: 0.7028 - val_loss: 0.4951 - val_accuracy: 0.7978


# Pre-training Generator

In [ ]:
%%capture
!pip install transformers
!pip install accelerate

In [ ]:
import re
import string
import pandas as pd
import numpy as np

In [ ]:
from transformers import Trainer, TextDataset, DataCollatorForLanguageModeling, TrainingArguments
from transformers import GPT2Tokenizer, PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast

In [ ]:
df = pd.read_csv("train.csv")

texts = df[df['target'] == 1]
tweets = texts['text']

In [ ]:
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    return text

In [ ]:
preprocessed_data = pd.DataFrame([preprocess_text(sentence) for sentence in tweets])
preprocessed_data = preprocessed_data.dropna()
preprocessed_data.to_csv('tweets.txt', index=False)
preprocessed_data.head()

,0
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...


In [ ]:
from os import path

def load_sentences(file_path):
    # Read the sentences from the text file
    with open(file_path, 'r', encoding='utf-8') as file:
        sentences = [line.strip() for line in file.readlines() if line.strip()]
    return sentences

def preprocess_sentences(sentences, tokenizer, block_size=128):
    # Tokenize and prepare the data for language modeling
    input_ids = []
    for sentence in sentences:
        tokenizer.pad_token = tokenizer.eos_token
        tokenized_sentence = tokenizer(sentence, add_special_tokens=True, truncation=True, padding='max_length', max_length=block_size)
        input_ids.append(tokenized_sentence['input_ids'])

    return input_ids

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):

    # если есть папка output_model, 'output_model', else 'model_name'
    if path.exists('output_model'):
        model_name = 'output_model'

    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    sentences = load_sentences(train_file_path)
    input_ids = preprocess_sentences(sentences, tokenizer)

    # Create a TextDataset
    dataset = TextDataset(tokenizer=tokenizer, file_path=train_file_path, block_size=128)

    # Create a DataCollatorForLanguageModeling
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    tokenizer.save_pretrained(output_dir)

    model = GPT2LMHeadModel.from_pretrained(model_name)

    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=overwrite_output_dir,
        per_device_train_batch_size=per_device_train_batch_size,
        num_train_epochs=num_train_epochs,
        save_steps=save_steps,
        report_to='none'
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    trainer.train()
    trainer.save_model()

In [ ]:
train_file_path = 'tweets.txt'
model_name = 'gpt2'
output_dir = 'output_model'
overwrite_output_dir = True
per_device_train_batch_size = 4
num_train_epochs = 20
save_steps = 1000

train(train_file_path, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs, save_steps)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,3.913600
1000,2.968300
1500,2.335800
2000,1.895600
2500,1.592000
3000,1.440300


In [ ]:
def generate_text(sequence, max_length):
    gpt_model = GPT2LMHeadModel.from_pretrained('output_model')
    gpt_tokenizer = GPT2Tokenizer.from_pretrained('output_model')
    ids = gpt_tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = gpt_model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=gpt_model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    return gpt_tokenizer.decode(final_outputs[0], skip_special_tokens=True)

def generate_batch_fake(n):
    fake = list()
    max_len = 50
    sequence = " "

    for i in range(n):
        fake.append(generate_text(sequence, max_len))

    return fake

# Test Them

In [ ]:
examples = pd.read_csv('train.csv')
examples.drop([examples.columns[i] for i in range(3)], axis=1, inplace=True)
examples = examples[examples['target'] == 1]
examples['text'].head()

15                  What's up man?
16                   I love fruits
17                Summer is lovely
18               My car is so fast
19    What a goooooooaaaaaal!!!!!!
Name: text, dtype: object

In [ ]:
def train_bert(sentences, labels, epochs=2):
    classifier.fit(x=sentences,
                   y=labels,
                   epochs=EPOCHS,
                   shuffle=True)

def test_bert(sentences):
    return np.argmax(classifier.predict(sentences), axis=1)

In [ ]:
import random

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    zipped = list(zip(a, b))
    random.shuffle(zipped)
    return zip(*zipped)

def training(N_EXAMPLES):
    sentences = generate_batch_fake(N_EXAMPLES)
    labels = [1 for i in range(len(sentences))]

    # save generated sentences
    generated = set(sentences)
    sentences.extend(examples['text'].sample(n=N_EXAMPLES))
    labels.extend([0 for i in range(len(sentences) - len(labels))])

    # shuffle
    # sentences, labels = unison_shuffled_copies(sentences, labels)

    # propagate through test
    values = test_bert(sentences)
    # sort out the ones that are fake, but classified as true
    tricky = [sentences[i] for i in range(len(values)) if values[i] > labels[i]]

    # train classifier for one epoch
    train_bert(sentences, labels, 2)

    if len(tricky) == 0:
        return

    file_path = "tweets_supervision.txt"
    # Open the file for writing
    with open(file_path, 'w') as f:
        # Write each sentence in 'tricky' list to the file
        for sentence in tricky:
            f.write(sentence + '\n')

    train_file_path = 'tweets_supervision.txt'
    model_name = 'gpt2'  # or other GPT-2 variants
    output_dir = 'output_model'
    overwrite_output_dir = True
    per_device_train_batch_size = len(tricky)
    num_train_epochs = 2
    save_steps = 1

    train(train_file_path, model_name, output_dir, overwrite_output_dir, per_device_train_batch_size, num_train_epochs, save_steps)

In [ ]:
training(1)